### PROJECT 2 - Artificial Vision

In [ ]:
import cv2
import numpy as np
from skimage.feature import graycomatrix, graycoprops 
from scipy.stats import skew, multivariate_normal      
import pandas as pd                 

### ----------------------------------------------------------------------------------------------------------------------------------------------------------------- ###
### --------------------------- Image acquisition ------------------------------------------------------------------------------------------------------------------- ###
### ----------------------------------------------------------------------------------------------------------------------------------------------------------------- ###

images_paths = [
    r"imgs\nuts_cam2_1.bmp",
    r"imgs\nuts_cam2_2.bmp",
    r"imgs\nuts_cam2_3.bmp",
    r"imgs\nuts_cam2_4.bmp",
    r"imgs\nuts_cam2_5.bmp",
    r"imgs\nuts_cam2_6.bmp",
]

img_path = r"imgs\nuts_cam2_5.bmp"



original_img = cv2.imread(img_path)
kernel_size = 5

# 1. Convert the image to grayscale
gray_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2GRAY)

# 2. Apply median filtering to reduce noise (kernel size 5 can be adjusted as needed)
median_img = cv2.medianBlur(gray_img, kernel_size)

# 3. Thresholding: apply a binary threshold to the median-filtered grayscale image
if len(median_img.shape) == 3:
    image = cv2.cvtColor(median_img, cv2.COLOR_BGR2GRAY)
otsu_threshold, binary_img = cv2.threshold(median_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Find contours in the inverted image
contours, _ = cv2.findContours(binary_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Count the number of objects found (each contour represents an object)
object_count = len(contours)
print("Number of objects found:", object_count)

### ----------------------------------------------------------------------------------------------------------------------------------------------------------------- ###
### --------------------------- Feature Extraction ------------------------------------------------------------------------------------------------------------------ ###
### ----------------------------------------------------------------------------------------------------------------------------------------------------------------- ###

# Container for storing features for each detected object
features = []

for i, contour in enumerate(contours):
    # --- Feature 1: Area --------------------------------
    area = cv2.contourArea(contour)

    # --- Feature 2: Perimeter --------------------------------
    perimeter = cv2.arcLength(contour, True)
    
    # --- Feature 3: Centroid --------------------------------
    M = cv2.moments(contour)
    if M["m00"] != 0:
        centroid = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
    else:
        centroid = (0, 0)
    
    # --- Feature 4: Average Color --------------------------------
    mask = np.zeros(binary_img.shape, dtype="uint8")
    cv2.drawContours(mask, [contour], -1, 255, -1)
    mean_color = cv2.mean(original_img, mask=mask)[:3]  # (B, G, R)
    
    # ------------------------------
    # Advanced Shape Features
    # ------------------------------

    # Bounding box and derived measurements
    x, y, w, h = cv2.boundingRect(contour)
    aspect_ratio = float(w) / h if h != 0 else 0
    extent = float(area) / (w * h) if (w * h) != 0 else 0
    circularity = (4 * np.pi * area) / (perimeter ** 2) if perimeter != 0 else 0
    
    # --- Feature 5: Hu Moment --------------------------------
    hu_moments = cv2.HuMoments(M).flatten()
    
     # --- Feature 6: Major and Minor Axis --------------------------------
    if len(contour) >= 5:
        ellipse = cv2.fitEllipse(contour)
        # ellipse returns ((xc, yc), (MA, ma), angle)
        major_axis = max(ellipse[1])
        minor_axis = min(ellipse[1])
    else:
        major_axis, minor_axis = None, None

    # --- Feature 7: Color Moments (per channel) --------------------------------
    color_moments = {}
    for idx, channel in enumerate(cv2.split(original_img)):
        # Extract pixel values within the object mask
        channel_vals = channel[mask == 255]
        if channel_vals.size > 0:
            channel_mean = np.mean(channel_vals)
            channel_std = np.std(channel_vals)
            channel_skew = skew(channel_vals.astype(np.float64))
        else:
            channel_mean, channel_std, channel_skew = 0, 0, 0
        color_moments[f'channel_{idx}'] = {
            'mean': channel_mean,
            'std': channel_std,
            'skewness': channel_skew
        }
    
   # --- Feature 8: Texture Features --------------------------------

    # Extract the Region Of Interest (ROI) from the grayscale image
    roi_gray = gray_img[y:y+h, x:x+w]
    # Also crop the corresponding mask region
    roi_mask = mask[y:y+h, x:x+w]
    
    # Set background (where mask==0) to 0 in the ROI and quantize the grayscale ROI
    roi_object = roi_gray.copy()
    roi_object[roi_mask == 0] = 0
    # Quantize to reduce the number of gray levels (here 16 levels)
    roi_quant = (roi_object // 16).astype(np.uint8)
    
    # Compute the gray level co-occurence matrix (GLCM) for a single distance & angle
    glcm = graycomatrix(roi_quant, distances=[1], angles=[0], levels=16, symmetric=True, normed=True)
    texture_features = {
        'contrast': graycoprops(glcm, 'contrast')[0, 0],
        'correlation': graycoprops(glcm, 'correlation')[0, 0],
        'energy': graycoprops(glcm, 'energy')[0, 0],
        'homogeneity': graycoprops(glcm, 'homogeneity')[0, 0]
    }
    
    # ------------------------------
    # Store All Features in Dictionary
    # ------------------------------
    feat = {
        "object": i + 1,
        "area": area,
        "perimeter": perimeter,
        "centroid": centroid,
        "mean_color": mean_color,
        # Advanced shape features
        "aspect_ratio": aspect_ratio,
        "extent": extent,
        "circularity": circularity,
        "hu_moments": hu_moments.tolist(),
        "major_axis": major_axis,
        "minor_axis": minor_axis,
        "color_moments": color_moments,
        "texture": texture_features,
        "bbox": (x, y, w, h)  
    }
    
    features.append(feat)
    
    # For visualization: Draw bounding box and centroid on the original image.
    cv2.rectangle(original_img, (x, y), (x+w, y+h), (0, 255, 0), 2)
    cv2.circle(original_img, centroid, 3, (0, 0, 255), -1)

# ------------------------------
# Output: Print Extracted Features for Each Object
# ------------------------------
for feat in features:
    print("Object {}:".format(feat["object"]))
    print("  Area: {:.2f}, Perimeter: {:.2f}".format(feat["area"], feat["perimeter"]))
    print("  Centroid: {}".format(feat["centroid"]))
    print("  Mean Color (BGR): {}".format(feat["mean_color"]))
    print("  Aspect Ratio: {:.2f}, Extent: {:.2f}, Circularity: {:.2f}".format(
        feat["aspect_ratio"], feat["extent"], feat["circularity"]))
    print("  Hu Moments: {}".format(feat["hu_moments"]))
    if feat["major_axis"] is not None:
        print("  Major Axis: {:.2f}, Minor Axis: {:.2f}".format(feat["major_axis"], feat["minor_axis"]))
    print("  Color Moments (per channel):")
    for ch, stats in feat["color_moments"].items():
        print("    {}: Mean = {:.2f}, Std = {:.2f}, Skewness = {:.2f}".format(
            ch, stats['mean'], stats['std'], stats['skewness']))
    print("  Texture Features (GLCM):", feat["texture"])
    print("\n---------------------\n")

# ------------------------------
# Display the Image with Overlaid Features
# ------------------------------
cv2.imshow("Detected Objects with Advanced Features", original_img)
cv2.waitKey(0)
cv2.destroyAllWindows()


### ----------------------------------------------------------------------------------------------------------------------------------------------------------------- ###
### --------------------------- Feature Selection  ------------------------------------------------------------------------------------------------------------------ ###
### ----------------------------------------------------------------------------------------------------------------------------------------------------------------- ###


# Build a feature matrix by “flattening” the extracted features.
feature_vectors = []
for feat in features:
    vector = {
       'area': feat['area'],
       'perimeter': feat['perimeter'],
       'aspect_ratio': feat['aspect_ratio'],
       'extent': feat['extent'],
       'circularity': feat['circularity'],
       'hu_moment_0': feat['hu_moments'][0],
        'hu_moment_1': feat['hu_moments'][1],
        'hu_moment_2': feat['hu_moments'][2],
        'hu_moment_3': feat['hu_moments'][3],
        'hu_moment_4': feat['hu_moments'][4],
        'hu_moment_5': feat['hu_moments'][5],
        'hu_moment_6': feat['hu_moments'][6],
       'major_axis': feat['major_axis'] if feat['major_axis'] is not None else 0,
       'minor_axis': feat['minor_axis'] if feat['minor_axis'] is not None else 0,
       'color_mean_0': feat['color_moments']['channel_0']['mean'],
       'color_mean_1': feat['color_moments']['channel_1']['mean'],
       'color_mean_2': feat['color_moments']['channel_2']['mean'],
       'texture_contrast': feat['texture']['contrast'],
       'texture_correlation': feat['texture']['correlation'],
       'texture_energy': feat['texture']['energy'],
       'texture_homogeneity': feat['texture']['homogeneity']
    }
    feature_vectors.append(vector)

# Create a DataFrame for ease of manipulation and visualization
df_features = pd.DataFrame(feature_vectors)
print("\nFeature Matrix (first 5 objects):")
print(df_features.head())

# Compute the correlation matrix among features to determine redundancy
corr_matrix = df_features.corr()
print("\nCorrelation Matrix:")
print(corr_matrix)

# Define a function for correlation-based feature selection
def select_features_by_correlation(corr_matrix, threshold=0.9):
    """
    Returns a list of feature names to drop when the absolute correlation 
    between features is greater than the specified threshold.
    """
    # Only consider the upper triangle of the correlation matrix
    upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper_tri.columns if any(upper_tri[column].abs() > threshold)]
    return to_drop

# Choose a threshold: features with correlation above this threshold are considered redundant.
features_to_drop = select_features_by_correlation(corr_matrix, threshold=0.9)
print("\nFeatures to drop due to high correlation (threshold = 0.9):")
print(features_to_drop)

# Drop the redundant features from the matrix
selected_features = df_features.drop(columns=features_to_drop)
print("\nSelected features after correlation-based selection (first 5 objects):")
print(selected_features.head())

### ----------------------------------------------------------------------------------------------------------------------------------------------------------------- ###
### --------------------------- Manual Labeling via GUI    ---------------------------------------------------------------------------------------------------------- ###
### ----------------------------------------------------------------------------------------------------------------------------------------------------------------- ###


# Create a list to hold labels for each object. Default is 'B' (not selected).
manual_labels = ['B'] * len(features)

# Create a copy of the original image with bounding boxes for labeling.
image_with_boxes = original_img.copy()

# Draw each object's bounding box with its index (for reference)
for feat in features:
    x, y, w, h = feat['bbox']
    cv2.rectangle(image_with_boxes, (x, y), (x+w, y+h), (0, 255, 0), 2)
    cv2.putText(image_with_boxes, str(feat['object']), (x, y-5),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

# Global variable to store manual labels.
# (We already have manual_labels list above.)
def click_event(event, x, y, flags, param):
    global manual_labels, features, image_with_boxes
    if event == cv2.EVENT_LBUTTONDOWN:
        for i, feat in enumerate(features):
            bx, by, bw, bh = feat['bbox']
            if bx <= x <= bx+bw and by <= y <= by+bh:
                # Mark this object as class 'A'
                manual_labels[i] = 'A'
                # Visualize the selection in red.
                cv2.rectangle(image_with_boxes, (bx, by), (bx+bw, by+bh), (0, 0, 255), 2)
                cv2.putText(image_with_boxes, "A", (bx, by-5),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                cv2.imshow("Labeling", image_with_boxes)
                break

cv2.namedWindow("Labeling")
cv2.setMouseCallback("Labeling", click_event)

print("Displaying the full image. Click on boxes that are Class A. When done, press 'q' to finish labeling.")
while True:
    cv2.imshow("Labeling", image_with_boxes)
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):  # Press 'q' to finish the labeling process.
        break
cv2.destroyWindow("Labeling")

print("\nManual labels assigned:")
for i, lbl in enumerate(manual_labels):
    print(f"Object {i+1}: {lbl}")

### ----------------------------------------------------------------------------------------------------------------------------------------------------------------- ###
### --------------------------- Classifier Implementation  ---------------------------------------------------------------------------------------------------------- ###
### ----------------------------------------------------------------------------------------------------------------------------------------------------------------- ###

df_features['class'] = manual_labels

print("\nSelected features with manual class labels:")
# Here, ensure the order in 'selected_features' is the same as in df_features.
selected_features = selected_features.copy()  # Create a copy to modify
selected_features['class'] = df_features['class']
print(selected_features.head())

# For this example, we use the entire data as our training set.
train_data = selected_features

# For each class, calculate the mean vector, covariance matrix, and prior probability.
classes = train_data['class'].unique()
class_params = {}
for cls in classes:
    group = train_data[train_data['class'] == cls]
    # Remove the 'class' column to get only the numeric features.
    feature_cols = group.columns.drop('class')
    mean_vec = group[feature_cols].mean().values
    cov_mat = group[feature_cols].cov().values
    prior = len(group) / len(train_data)
    class_params[cls] = (mean_vec, cov_mat, prior)

# Define a function that classifies a sample based on the multivariate Gaussian density.
def classify_sample(x, class_params):
    probabilities = {}
    for cls, (mean_vec, cov_mat, prior) in class_params.items():
        cov_mat_adjusted = cov_mat + np.eye(cov_mat.shape[0]) * 1e-6
        likelihood = multivariate_normal.pdf(x, mean=mean_vec, cov=cov_mat_adjusted, allow_singular=True)
        posterior = likelihood * prior
        probabilities[cls] = posterior
    predicted_class = max(probabilities, key=probabilities.get)
    return predicted_class, probabilities

# Classify each sample in our training dataset and print the results.
predictions = []
feature_cols = train_data.columns.drop('class')
for idx, row in train_data.iterrows():
    x = row[feature_cols].values
    pred_class, probs = classify_sample(x, class_params)
    predictions.append(pred_class)
    print(f"Sample {idx}: True class: {row['class']}, Predicted: {pred_class}, Posterior probabilities: {probs}")

# For evaluation, compute the accuracy on this (training) set.
train_data['predicted'] = predictions
accuracy = (train_data['class'] == train_data['predicted']).mean()
print(f"\nClassification accuracy on training set: {accuracy * 100:.2f}%")

Number of objects found: 8
Object 1:
  Area: 7595.00, Perimeter: 339.08
  Centroid: (285, 648)
  Mean Color (BGR): (214.67213326446281, 226.26330061983472, 231.54764979338844)
  Aspect Ratio: 1.10, Extent: 0.78, Circularity: 0.83
  Hu Moments: [0.16210750633072468, 0.00021990002842822536, 8.980517180104648e-05, 3.6695107615721506e-07, 3.5194655234613833e-13, 4.665466331748814e-09, 2.0768980934224168e-12]
  Major Axis: 103.07, Minor Axis: 95.28
  Color Moments (per channel):
    channel_0: Mean = 214.67, Std = 30.44, Skewness = -1.60
    channel_1: Mean = 226.26, Std = 14.24, Skewness = -2.44
    channel_2: Mean = 231.55, Std = 16.47, Skewness = -2.82
  Texture Features (GLCM): {'contrast': 1.993950771798081, 'correlation': 0.9666834360456708, 'energy': 0.6684010818411942, 'homogeneity': 0.9365732386879052}

---------------------

Object 2:
  Area: 8991.50, Perimeter: 372.63
  Centroid: (648, 529)
  Mean Color (BGR): (191.25177498634625, 216.43255051884216, 235.91327143637358)
  Aspect 